In [1]:
%reset
import pandas as pd
import numpy as np
from datetime import datetime
import h5py
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.vector_ar import vecm
from statsmodels.tsa.vector_ar.vecm import select_coint_rank
from statsmodels.tsa.stattools import coint
from sklearn.preprocessing import StandardScaler, MinMaxScaler

In [2]:
def df_creator(file_path):                                                                   #Create an orderbook dataframe for every month/seperate h5 file.
    data_file = h5py.File(file_path,'r')                                                     #Use h5py package to read in the h5 file.
    df_total = pd.DataFrame(data = np.array(data_file['/data/block0_values']))          #Create a DataFrame containing all the values in the orderbook
    df_total.columns = np.array(data_file['/data/block0_items'])   
                         #Fill in the column names using the block0_items: Example: b'bidsP:0|binance-perps|XRP-USDT_PS'
    df_index= pd.DataFrame(data = np.array(data_file['/data/axis1']))       
    df_index.columns = ["orderbook_index"]
    df_total['timestamp'] = df_index.orderbook_index
    df_total.index = pd.to_datetime(df_total.timestamp,unit='ns')
    df_total = df_total.drop(columns=['timestamp'])
    return df_total       

In [3]:
file_path_ob = "/Users/markdekwaasteniet/Documents/Master Finance/Master Thesis/BTC_Data/mark_ob.h5" 
file_path_trades = "/Users/markdekwaasteniet/Documents/Master Finance/Master Thesis/BTC_Data/mark_trades.h5"
orderbook_data = df_creator(file_path_ob)
trades_data = df_creator(file_path_trades)

columns_dict = {'volume':1, 'type':2, 'price':0}
df_trade_volume = trades_data.copy()
columns_dict = {'volume':1, 'type':2, 'price':0}
for c, i in columns_dict.items():
    df_trade_volume[c] = np.nan
    df_trade_volume[c] = df_trade_volume.iloc[:,i].values

df_trade_volume = df_trade_volume.iloc[:,3:]


In [850]:
# def order_placement(df_orders_placed, df_trades_executed, c):

#     for i in range(len(df_orders_placed)):
#         df_orders_placed.loc[df_orders_placed.index[i], 'sold_orders'] = df_trades_executed[c].dropna().loc[str(df_orders_placed.timestamp.iloc[i]):str(df_orders_placed.timestamp_shift.iloc[i])].values.sum()
#         sold_order = df_orders_placed.loc[df_orders_placed.index[i], 'sold_orders']
#         cancelled_order = df_orders_placed.loc[df_orders_placed.index[i], 'quantity_change']


#         if (cancelled_order < 0) and (abs(cancelled_order) > abs(sold_order)):
#             df_orders_placed.loc[df_orders_placed.index[i], 'order_subtraction'] = cancelled_order
#         elif (cancelled_order < 0) and (abs(cancelled_order) < abs(sold_order)):
#             df_orders_placed.loc[df_orders_placed.index[i], 'order_subtraction']  = -sold_order
#         else:
#             df_orders_placed.loc[df_orders_placed.index[i], 'order_subtraction']  = -sold_order
        
#         if (cancelled_order > 0) or ((cancelled_order < 0) and (abs(cancelled_order) < abs(sold_order))):
#             df_orders_placed.loc[df_orders_placed.index[i], 'order_placed']  = sold_order + cancelled_order
#         else:
#             df_orders_placed.loc[df_orders_placed.index[i], 'order_placed'] = cancelled_order
#     return df_orders_placed

In [4]:
def order_placement(df_orders_placed):
    df_orders_placed.loc[(df_orders_placed['sold_orders'] > 0), 'order_subtraction'] = -df_orders_placed['sold_orders']

    df_orders_placed.loc[(df_orders_placed['quantity_change'] < 0) & (abs(df_orders_placed['quantity_change']) > abs(df_orders_placed['sold_orders'])), 'order_subtraction'] = df_orders_placed['quantity_change']
    df_orders_placed['order_subtraction'] = df_orders_placed['order_subtraction'].fillna(0)
    
    df_orders_placed.loc[(df_orders_placed['quantity_change'] > 0), 'order_placed'] = (df_orders_placed['sold_orders'] + df_orders_placed['quantity_change'])
    df_orders_placed.loc[((df_orders_placed['quantity_change'] < 0) & (abs(df_orders_placed['quantity_change']) < abs(df_orders_placed['sold_orders']))), 'order_placed'] = (df_orders_placed['sold_orders'] + df_orders_placed['quantity_change'])
    return df_orders_placed

In [105]:
def threshold_finder(placed_order, df_orders_placed):
    threshold = df_orders_placed.loc[str(placed_order.timestamp)].order_quantity
    df_cumsum = df_orders_placed.loc[str(placed_order.timestamp):].iloc[1:,:]
    # df_cumsum['cumsum_res'] = df_cumsum['order_subtraction'].cumsum()
    # df_cumsum.loc[(df_cumsum['sold_orders'] <= 0), 'cumsum_res'] = 0
    # try:
    #     #df_orders_placed.loc[df_orders_placed.index[placed_order], 'timestamp_execution'] = df_cumsum[df_cumsum['cumsum_res'].lt(threshold)].index[0]
    #     return df_cumsum[df_cumsum['cumsum_res'].lt(threshold)].index[0]
    # except:
    #     return np.nan
    running_sum = 0
    for future_date in range(1, len(df_cumsum)):
        running_sum += abs(df_cumsum.loc[df_cumsum.index[future_date], 'order_subtraction'])
        try:
            if running_sum >= threshold:
                if df_cumsum.loc[df_cumsum.index[future_date], 'sold_orders'] > 0:
                    #df_orders_placed.loc[df_orders_placed.index[placed_order], 'timestamp_execution'] = df_cumsum.loc[df_cumsum.index[future_date], 'timestamp']
                    return df_cumsum.loc[df_cumsum.index[future_date], 'timestamp']
                    break
        except:
            return np.nan

    # threshold = -df_orders_placed.loc[df_orders_placed.index[placed_order], 'order_quantity']
    # df_cumsum = df_orders_placed.iloc[(placed_order+1):,:]
    # df_cumsum['cumsum_res'] = df_cumsum['order_subtraction'].cumsum()
    # df_cumsum.loc[(df_cumsum['sold_orders'] <= 0), 'cumsum_res'] = 0
    # try:
    #     #df_orders_placed.loc[df_orders_placed.index[placed_order], 'timestamp_execution'] = df_cumsum[df_cumsum['cumsum_res'].lt(threshold)].index[0]
    #     return df_cumsum[df_cumsum['cumsum_res'].lt(threshold)].index[0]
    # except:
    #     return np.nan
        #print(f"Order at {str(df_orders_placed.loc[df_orders_placed.index[placed_order], 'timestamp'])} not yet executed")
    return


In [106]:
def execution_date(df_orders_placed):

    mask = (df_orders_placed['order_placed'] > 0)
    df_orders_placed.loc[mask, 'timestamp_execution'] = df_orders_placed.loc[mask].apply(lambda placed_order: threshold_finder(placed_order, df_orders_placed), axis=1)

    # for placed_order in range(len(df_orders_placed)):
    #     if df_orders_placed.loc[df_orders_placed.index[placed_order], 'order_placed'] > 0:
    #         threshold = -df_orders_placed.loc[df_orders_placed.index[placed_order], 'order_quantity']
    #         df_cumsum = df_orders_placed.iloc[(placed_order+1):,:]
    #         # df_cumsum['cumsum_res'] = df_cumsum['order_subtraction'].cumsum()
    #         # df_cumsum.loc[(df_cumsum['sold_orders'] <= 0), 'cumsum_res'] = 0
    #         # try:
    #         #     df_orders_placed.loc[df_orders_placed.index[placed_order], 'timestamp_execution'] = df_cumsum[df_cumsum['cumsum_res'].lt(threshold)].index[0]
    #         # except:
    #         #     print(f"Order at {str(df_orders_placed.loc[df_orders_placed.index[placed_order], 'timestamp'])} not yet executed")
    #         # df_orders_placed.loc[df_orders_placed.index[placed_order], 'timestamp_execution'] = df_cumsum[df_cumsum['cumsum_res'].le(threshold)].index
            
    #         running_sum = 0
    #         for future_date in range(1, len(df_cumsum)):
    #             running_sum += abs(df_cumsum.loc[df_cumsum.index[future_date], 'order_subtraction'])
    #             if running_sum >= threshold:
    #                 if df_cumsum.loc[df_cumsum.index[future_date], 'sold_orders'] > 0:
    #                     df_orders_placed.loc[df_orders_placed.index[placed_order], 'timestamp_execution'] = df_cumsum.loc[df_cumsum.index[future_date], 'timestamp']
    #                     break 
    return df_orders_placed

In [107]:
def calc_execution_time(df_trades, df_orderbook):
    df_check = {}
    types_dict = {"bid":-1, "ask":1}
    columns_list = ['sold_orders', 'order_subtraction', 'order_placed', 'timestamp_execution']
    
    df_trades_executed = pd.DataFrame(index = df_trades.index)

    for c, item in types_dict.items():
        df_trades_executed[c] = df_trade_volume.loc[df_trade_volume['type'] == item].volume
        df_orders_placed = df_orderbook.iloc[:,((item+1)*10 + 10)].diff()
        custom_dates = orderbook_data.index
        custom_sum = df_trades_executed[c].groupby(custom_dates[custom_dates.searchsorted(df_trades_executed[c].index)]).sum()
        df_orders_placed = df_orders_placed.to_frame().join(custom_sum.to_frame())
        df_orders_placed = df_orders_placed.reset_index()
        df_orders_placed['timestamp_shift'] = df_orders_placed.timestamp.shift(-1).dropna()

        df_orders_placed.rename(columns={df_orders_placed.columns[1]: "quantity_change" }, inplace = True)
        df_orders_placed = df_orders_placed.iloc[:(len(df_orderbook)-1),:]
        for column in columns_list:
            df_orders_placed[column] = np.nan
        
        df_orders_placed['sold_orders'] = df_orders_placed[c]
        df_orders_placed['sold_orders'] = df_orders_placed['sold_orders'].fillna(0)
        df_orders_placed = df_orders_placed.drop(columns=c)
        df_orders_placed = order_placement(df_orders_placed)
        
        df_orders_placed = df_orders_placed.set_index(df_orders_placed.timestamp)
        df_orders_placed['order_quantity'] = df_orderbook.iloc[:,((item+1)*10 + 10)]

        df_orders_placed = execution_date(df_orders_placed)

        df_orders_placed = df_orders_placed.join(df_orderbook.iloc[:,((item+1)*10)])
        df_orders_placed.rename(columns={df_orders_placed.columns[8]: "price"}, inplace = True)
        df_orders_placed = df_orders_placed.iloc[:,5:]
        df_check[c] = df_orders_placed
    df_merged = df_check['bid'].join(df_check['ask'], lsuffix='_bid', rsuffix='_ask')
    
    return df_merged

In [108]:
df = calc_execution_time(df_trade_volume, orderbook_data)

In [110]:
df_check = df
df_check

,order_placed_bid,timestamp_execution_bid,order_quantity_bid,price_bid,order_placed_ask,timestamp_execution_ask,order_quantity_ask,price_ask
timestamp,,,,,,,,
2022-03-27 10:30:00.020911,NaN,NaN,20.533,44564.4,NaN,NaN,0.201,44564.5
2022-03-27 10:30:00.060140,0.500,2022-03-27 10:30:06.945889,21.022,44564.4,0.002,2022-03-27 10:30:02.123529,0.203,44564.5
2022-03-27 10:30:00.092865,NaN,NaN,21.022,44564.4,NaN,NaN,0.203,44564.5
2022-03-27 10:30:00.125876,NaN,NaN,21.022,44564.4,NaN,NaN,0.203,44564.5
2022-03-27 10:30:00.159058,NaN,NaN,21.022,44564.4,NaN,NaN,0.203,44564.5
...,...,...,...,...,...,...,...,...
2022-03-27 10:39:59.820584,NaN,NaN,1.686,44551.6,0.001,NaN,0.001,44551.7
2022-03-27 10:39:59.853315,4.146,NaN,5.832,44551.6,0.491,NaN,0.492,44551.7
2022-03-27 10:39:59.886823,1.400,NaN,7.232,44551.6,NaN,NaN,0.492,44551.7


In [463]:
# df_trade_volume = df_trades.copy()
# columns_dict = {'volume':1, 'type':2, 'price':0}
# for c, i in columns_dict.items():
#     df_trade_volume[c] = np.nan
#     df_trade_volume[c] = df_trade_volume.iloc[:,i].values

# df_trade_volume = df_trade_volume.iloc[:,3:]
# df_bid_trade_volume = df_trade_volume.loc[df_trade_volume['type'] == -1]
# df_ask_trade_volume = df_trade_volume.loc[df_trade_volume['type'] == 1]
# df_bid_trade_volume

In [464]:
# df_orders_placed = orderbook_data.iloc[:,10].diff()
# df_orders_placed = df_orders_placed.reset_index()
# df_orders_placed['timestamp_shift'] = df_orders_placed.Timestamp.shift(-1).dropna()
# df_orders_placed.rename(columns={df_orders_placed.columns[1]: "bid_quantity_change" }, inplace = True)
# df_orders_placed = df_orders_placed.iloc[:1773,:]
# df_orders_placed['sold_orders'] = np.nan
# df_orders_placed['order_placed'] = np.nan
# df_orders_placed['order_subtraction'] = np.nan
# for i in range(len(df_orders_placed)):
#     print(df_bid_trade_volume['volume'])
#     df_orders_placed.iloc[i,3] = df_bid_trade_volume['volume'].loc[str(df_orders_placed.Timestamp.iloc[i]):str(df_orders_placed.timestamp_shift.iloc[i])].values.sum()
#     if i >= 1:
#         if (df_orders_placed.iloc[i,1] < 0) and (abs(df_orders_placed.iloc[i,1]) > abs(df_orders_placed.iloc[(i-1),3])):
#             df_orders_placed.iloc[i,5] = df_orders_placed.iloc[i,1]
#         elif (df_orders_placed.iloc[i,1] < 0) and (abs(df_orders_placed.iloc[i,1]) < abs(df_orders_placed.iloc[(i-1),3])):
#             df_orders_placed.iloc[i,5] = -df_orders_placed.iloc[(i-1),3]
#         else:
#             df_orders_placed.iloc[i,5] = -df_orders_placed.iloc[(i-1),3]
        
#         if df_orders_placed.iloc[i,1] > 0:
#             df_orders_placed.iloc[i,4] = df_orders_placed.iloc[(i-1),3] + df_orders_placed.iloc[(i),1]
#         elif (df_orders_placed.iloc[i,1] < 0) and (abs(df_orders_placed.iloc[i,1]) < abs(df_orders_placed.iloc[(i-1),3])):
#             df_orders_placed.iloc[i,4] = df_orders_placed.iloc[(i-1),3] + df_orders_placed.iloc[i,1] 
#         else:
#             df_orders_placed.iloc[i,4] = df_orders_placed.iloc[i,1]
    
# df_orders_placed['sold_orders'] = df_orders_placed['sold_orders'].shift(1)
# df_orders_placed = df_orders_placed.set_index(df_orders_placed.Timestamp)
# df_orders_placed['OrderQuantity'] = orderbook_data.iloc[:,10]
# df_orders_placed["timestamp_execution"] = np.nan

# for placed_order in range(len(df_orders_placed)):
#     if df_orders_placed.iloc[placed_order,4] > 0:
#         threshold = df_orders_placed.iloc[placed_order,6]
#         df_cumsum = df_orders_placed.iloc[placed_order:,:]
#         running_sum = 0
#         for i in range(len(df_cumsum)):
#             if df_cumsum.iloc[i,5] < 0:
#                 running_sum += abs(df_cumsum.iloc[i,5])

#             if running_sum > threshold:
#                 if df_cumsum.iloc[i,3] != 0:
#                     df_orders_placed.iloc[placed_order,7] = df_cumsum.iloc[(i-1),2]
#                     break

# df_orders_placed = df_orders_placed.join(orderbook_data.iloc[:,0])


In [465]:
# df_orders_placed = orderbook_data.iloc[:,30].diff()
# df_orders_placed = df_orders_placed.reset_index()
# df_orders_placed['timestamp_shift'] = df_orders_placed.Timestamp.shift(-1).dropna()
# df_orders_placed.rename(columns={df_orders_placed.columns[1]: "bid_quantity_change" }, inplace = True)
# df_orders_placed = df_orders_placed.iloc[:1773,:]
# df_orders_placed['sold_orders'] = np.nan
# df_orders_placed['order_placed'] = np.nan
# df_orders_placed['order_subtraction'] = np.nan
# for i in range(len(df_orders_placed)):
#     df_orders_placed.iloc[i,3] = df_ask_trade_volume['volume'].loc[str(df_orders_placed.Timestamp.iloc[i]):str(df_orders_placed.timestamp_shift.iloc[i])].values.sum()
#     if i >= 1:
#         if (df_orders_placed.iloc[i,1] < 0) and (abs(df_orders_placed.iloc[i,1]) > abs(df_orders_placed.iloc[(i-1),3])):
#             df_orders_placed.iloc[i,5] = df_orders_placed.iloc[i,1]
#         elif (df_orders_placed.iloc[i,1] < 0) and (abs(df_orders_placed.iloc[i,1]) < abs(df_orders_placed.iloc[(i-1),3])):
#             df_orders_placed.iloc[i,5] = -df_orders_placed.iloc[(i-1),3]
#         else:
#             df_orders_placed.iloc[i,5] = -df_orders_placed.iloc[(i-1),3]
        
#         if df_orders_placed.iloc[i,1] > 0:
#             df_orders_placed.iloc[i,4] = df_orders_placed.iloc[(i-1),3] + df_orders_placed.iloc[(i),1]
#         elif (df_orders_placed.iloc[i,1] < 0) and (abs(df_orders_placed.iloc[i,1]) < abs(df_orders_placed.iloc[(i-1),3])):
#             df_orders_placed.iloc[i,4] = df_orders_placed.iloc[(i-1),3] + df_orders_placed.iloc[i,1] 
#         else:
#             df_orders_placed.iloc[i,4] = df_orders_placed.iloc[i,1]
    
# df_orders_placed['sold_orders'] = df_orders_placed['sold_orders'].shift(1)
# df_orders_placed = df_orders_placed.set_index(df_orders_placed.Timestamp)
# df_orders_placed['OrderQuantity'] = orderbook_data.iloc[:,30]
# df_orders_placed["timestamp_execution"] = np.nan

# for placed_order in range(len(df_orders_placed)):
#     if df_orders_placed.iloc[placed_order,4] > 0:
#         threshold = df_orders_placed.iloc[placed_order,6]
#         df_cumsum = df_orders_placed.iloc[placed_order:,:]
#         running_sum = 0
#         for i in range(len(df_cumsum)):
#             if df_cumsum.iloc[i,5] < 0:
#                 running_sum += abs(df_cumsum.iloc[i,5])

#             if running_sum > threshold:
#                 if df_cumsum.iloc[i,3] != 0:
#                     df_orders_placed.iloc[placed_order,7] = df_cumsum.iloc[(i-1),2]
#                     break

# df_orders_placed_ask = df_orders_placed.join(orderbook_data.iloc[:,20])